In [1]:
from google.colab import drive
import os
CODE_PATH='/content/drive/My Drive/TCC/Codes/Codigos Atualizados/'
DATA_PATH = '/content/drive/My Drive/TCC/EMG_Database/'
drive.mount('/content/drive', force_remount=True)
if not os.getcwd() == CODE_PATH:
  os.chdir(CODE_PATH)
# google.colab.drive module has a recently-added flush_and_unmount()

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from dataset import Dataset
from dataset import load_saved_windows
from feature_extraction import SignalParser
import os
import time

Using TensorFlow backend.


In [0]:
#from dataset import load_data_and_save_windows
from preprocessing import envelope, dwt_filter, band_pass
from feature_extraction import SignalParser
import os
from os.path import join
import numpy as np
import multiprocessing as mp
def create_dir(dirpath):
  if not os.path.exists(dirpath):
    os.mkdir(dirpath)
 
# Creating the Parser
sp = SignalParser(extractors=[['KRTT',
                               'SKWT',
                               'STDT',
                               'VART',
                               'MENT',
                               'ABMT',
                               'RMST',
                               'ZCCT',
                               'WAPT',
                               'ENGT',
                               'PRCT',
                               'CORT',
                               'RENT',
                               'PSPT',
                               'WCXT',
                               'SPRT',
                               'DWET'],
                              ['SPCF',
                               'KRTF',
                               'SKWF',
                               'STDF',
                               'VARF',
                               'MENF',
                               'HSTF',
                               'PRCF',
                               'CORF',
#                                'KLDF'
                              ]])
first_time = time.time()
strides = np.flip([128, 256, 256, 512])
windows_sizes = np.flip([2**8, 2**9, 2**10, 2**11])
path = join(DATA_PATH, "Features_Files")

for use_envelope in ["enveloped","raw"]: # For te enveloped and raw data
  print(use_envelope)
  for pure_samples in ["pure","nonpure"]: # For the pure samples and nonpure data
    print(pure_samples)
    for stride, window_size in zip(strides, windows_sizes): # For every fold used
      print(f"FOLD {window_size}")
      start_iter_time = time.time()
      
      # Create the save folder
      joined_path = join(path, use_envelope, pure_samples, f"{window_size}_{stride}")
      try:
        os.makedirs(joined_path)
      except:
        pass
      # Get the data folder
      load_folder = joined_path.split('/')
      load_folder[load_folder.index("Features_Files")] = "Windows_Files"
      load_folder = "/".join(load_folder)
      
      print("Subs: ", end='')
      
      # Worker function for parsing and saving one subject
      def func(subject, is_enveloped=None):
        try:
          # If the feature file already exists, skip this subject
          time_start = time.time()
          if os.path.isfile(join(joined_path, f"features_{subject}.npy")) or (not os.path.isfile(join(load_folder, f"data_{subject}.npy"))):
            print(f"{subject}X ")
            return
          print(f"{subject}_{int(is_enveloped=='enveloped')}start ",)
          
          # Load the windows files
          data, labels, ids = load_saved_windows(load_folder, subjects=[subject])
#           if is_enveloped=="raw":
#             # Band passing filter
#             print(f"{subject} filtering ",)
#             data = band_pass(data, axis=1)
#             # DWT filtering
#             print(f"{subject} denoising ",)
#             data = dwt_filter(data, axis=1)
          # Extract the features (1110)
          print(f"{subject} extracting ",)
          features, names = sp.extract(data, axis=1, features_and_names=True)
          # Save to save path
          print(f"{subject} saving ")
          np.save(os.path.join(joined_path, f"features_{subject}"), features)
          time_end = time.time()
          print(f"{subject} end (took {round((time_end-time_start)/60)}min) ")
          return None
        except:
          print(f"{subject} FAIL")
          return None

      # Throw 4 threads for parallel processing
      with mp.Pool(processes=mp.cpu_count()) as pool:
        pool.starmap(func, zip(range(1,41), [use_envelope]*40), chunksize=4)
      print()
      
      # Save features names using a dummy array
      _, names = sp.extract(np.random.rand(10,512,12), debug=False, features_and_names=True)
      with open(os.path.join(joined_path, "features_names.txt"), "w") as file:
        file.writelines("\n".join(names))

      drive.flush_and_unmount()
      drive.mount('/content/drive', force_remount=True)

    print(f"FOLD {pure_samples}: {time.time()-start_iter_time}s")

print(f"TOTAL TIME: {time.time()-first_time}s")

In [0]:
drive.flush_and_unmount()
drive.mount('/content/drive', force_remount=True)


In [0]:
#from dataset import load_data_and_save_windows
from preprocessing import envelope, band_pass, dwt_filter
import os
from os.path import join
import numpy as np
from preprocessing import envelope

def create_dir(dirpath):
  if not os.path.exists(dirpath):
    os.mkdir(dirpath)
  
first_time = time.time()
path = join(DATA_PATH, "Windows_Files")
strides = [128, 256, 256, 512]
windows_sizes = [2**8, 2**9, 2**10, 2**11]
for subject in range(32, 41):

  
  first_suject_time = time.time()
  
    
  print(f"SUBJECT {subject}")
  data = None

  for use_envelope in ["enveloped","raw"]:
    for pure_samples in ["pure","nonpure"]:
      for window_size, stride in zip(windows_sizes, strides):
        start_iter_time = time.time()
        joined_path = join(path, use_envelope, pure_samples, f"{window_size}_{stride}")
        try:
          os.makedirs(joined_path)
        except:
          pass
        get_dataset_kws = dict(window_size=window_size,
                               stride=stride,
                               pure_samples=True if pure_samples=="pure" else False,
#                                processing_func=envelope if use_envelope=="enveloped" else None,
                               save_memory=True)
        if not (os.path.isfile(join(joined_path, f"data_{subject}.npy")) and os.path.isfile(join(joined_path, f"labels_{subject}.npy")) and os.path.isfile(join(joined_path, f"ids_{subject}.npy"))):

          if data==None:
              data = Dataset(DATA_PATH)
              data.get_data(subjects=[subject], verbose=0)
              data[subject].emg = band_pass(data[subject].emg, axis=0)
              data[subject].emg = dwt_filter(data[subject].emg, axis=0)

              raw_emg = data[subject].emg
              enveloped_emg = envelope(data[subject].emg, axis=0)
          
          if use_envelope == "enveloped":
            data[subject].emg = enveloped_emg
          else:
            data[subject].emg = raw_emg
            
          windows, labels, ids = data.get_dataset(**get_dataset_kws)

          np.save(join(joined_path, f"data_{subject}.npy"), windows)
          np.save(join(joined_path, f"labels_{subject}.npy"), labels)
          np.save(join(joined_path, f"ids_{subject}.npy"), ids)
          print(f"TIME {use_envelope.upper()} {pure_samples.upper()} {window_size}: {time.time()-start_iter_time}s")
        else:
          print(f"{use_envelope.upper()} {pure_samples.upper()} {window_size}: ALREADY EXISTS")
  data == None
  print(f"SUBJECT {subject} TIME: {time.time()-first_suject_time}s")
print(f"TOTAL TIME: {time.time()-first_time}s")

SUBJECT 32
ENVELOPED PURE 256: ALREADY EXISTS
ENVELOPED PURE 512: ALREADY EXISTS
ENVELOPED PURE 1024: ALREADY EXISTS
ENVELOPED PURE 2048: ALREADY EXISTS
ENVELOPED NONPURE 256: ALREADY EXISTS
ENVELOPED NONPURE 512: ALREADY EXISTS
ENVELOPED NONPURE 1024: ALREADY EXISTS
ENVELOPED NONPURE 2048: ALREADY EXISTS
Acessing  /content/drive/My Drive/TCC/EMG_Database/DB2_s32.zip
TIME RAW PURE 256: 47.977609395980835s
TIME RAW PURE 512: 3.9417786598205566s
TIME RAW PURE 1024: 8.670864343643188s
TIME RAW PURE 2048: 6.3561742305755615s
TIME RAW NONPURE 256: 9.865298748016357s
TIME RAW NONPURE 512: 7.152770042419434s
TIME RAW NONPURE 1024: 10.368547201156616s
TIME RAW NONPURE 2048: 9.591524600982666s
SUBJECT 32 TIME: 113.00248956680298s
SUBJECT 33
Acessing  /content/drive/My Drive/TCC/EMG_Database/DB2_s33.zip
TIME ENVELOPED PURE 256: 64.76744985580444s
TIME ENVELOPED PURE 512: 5.019580125808716s
TIME ENVELOPED PURE 1024: 8.22701621055603s
TIME ENVELOPED PURE 2048: 5.35356879234314s
TIME ENVELOPED NONP

In [0]:
cd /

/


In [0]:
!du -a /content | sort -n -r | head -n 5


^C
